# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.16it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.18it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.72it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.42it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.40it/s]



...

 65%|██████▌   | 15/23 [00:04<00:01,  5.47it/s]

 74%|███████▍  | 17/23 [00:04<00:01,  5.43it/s]

 83%|████████▎ | 19/23 [00:04<00:00,  5.52it/s]

 91%|█████████▏| 21/23 [00:05<00:00,  5.55it/s]

100%|██████████| 23/23 [00:05<00:00,  4.08it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Nicole and I am a 4th grade teacher at a public elementary school. I have been teaching for 10 years and I absolutely love my job! I teach a range of subjects including reading, writing, social studies, and science.
I am excited to be a part of this community and look forward to sharing my teaching experiences and tips with you. I am passionate about creating engaging and fun lesson plans that cater to the needs of all learners. In this blog, I will be sharing my favorite lesson plans, classroom management ideas, and educational technology tips.
I also love connecting with other teachers and learning from their experiences, so please don
Prompt: The president of the United States is
Generated text:  elected to serve as the head of state and the head of government for a four-year term. The president is also the commander-in-chief of the armed forces and is responsible for setting national policy. The president is elected through the Electoral C

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Dan

 and

 I

 am

...

 biases

.

 This

 can

 lead

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Saeed and I am a software engineer with a passion for design and user experience. I am also the founder and lead developer of several applications, including our company’s flagship app, which is a leading productivity app in the Middle East. I enjoy creating clean and intuitive designs, as well as writing efficient and scalable code. My work experience ranges from freelancing to leading a team of developers, and I have a strong background in both design and development. I am also a certified Scrum Master and Agile practitioner, with experience in implementing Agile methodologies in different projects.
I have a strong background in design and development, and I am well-

Prompt: The capital of France is
Generated text: , of course, Paris, but many people know that it was actually established by the Gauls, a tribe of Celtic people. They called it Lutetia and it was a small fortified settlement. The Romans, who conquered the Gauls in the 1st cen

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Pat

rice

,

 and

...

 it

 accountable

 if

 we

 don

In [6]:
llm.shutdown()